In [1]:
# from network.Gamma2DCropPaste import StackTPredict, UNet, StackSequenceT , Gamma2DCropPaste, Gamma2DNet
from network.mapGeneratorNet import LSTMNet, Generator, GeneratorPredict

import torch
import numpy as np 
from common.readFiles import *
from matplotlib import pyplot as plt
from policy.PytorchTraining import PytorchRunner
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
map_x = 5
variables = ["source_pos", "source_region", "X_input", "source_concentration", 
             "water_flow_force", "source_inside", "distance_map", "source_detect",
             "source_distance", "agent_action", "agent_position", "centroids", "source_detect"]

datas = []
for map_x in [5, 10, 20]:
    path = "../output/data/processedData/reinforce/stochastic/{}/".format(map_x)
    datas.append(getData(path, variables) )

In [3]:
map_x = 5
env_arg = {
    "predict_map_shape": (map_x, map_x),
    
    "directions": torch.tensor([[0, -1], [1, 0], [0, 1], [-1, 0]]),
    "dt":5,
    
    "feature_dim":3,
}

model_arg = {
    "T":5,
    "k_tap":7,
    "T_range":5,
    "T_threshold": 0,
    "up" : 1,
    "skip_connection":False,
    "mu": 1,
}
runner_arg = {
    "new_loss" : True,
    "regularizer_constant": 0.01,
    "accuracy_range":0,
    "folder_path":"../output/ReportModel/stochastic/{}outOf{}/",
    "name":"LSTM"
}

In [4]:
LSTM_model = GeneratorPredict(env_arg, model_arg).to(DEVICE)
pytorch_total_params = sum(p.numel() for p in LSTM_model.parameters())
pytorch_total_params

73402

In [5]:
# T_thresholds = [0] #, 1, 1]
# T_ranges = [ 5] #, 20, 30]

env_arg["predict_map_shape"] = (5,5)
new_losses = [False]
regularizer = [0] #, 0.01, 0.1]
ups = [3, 2, 1]
ups = [1]
# T_thresholds = [1]
# T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]

for (i,up) in enumerate(ups):
    source_pos, source_region, X_input, source_concentration, water_flow_force, source_inside, distance_map, source_detect, source_distance, agent_action, agent_position, centroids, source_detect = datas[up-1]
    for new_loss in new_losses:
        for regularizer_constant in regularizer:
            model_arg['up'] = up
            runner_arg["regularizer_constant"] = regularizer_constant
            runner_arg["new_loss"] = new_loss
            runner_arg["folder_path"] = "../output/ReportModel/stochastic/reinforce/"
            # runner_arg["folder_path"] = "../output/ReportModel/upDown/evaluate/"
            runner_arg["name"] = "model_{}_{}_{}".format(up*5,new_loss, regularizer_constant)
            LSTM_model = GeneratorPredict(env_arg, model_arg).to(DEVICE)
            LSTM_runner = PytorchRunner(runner_arg=runner_arg)
            LSTM_runner.modelInitialize(LSTM_model)
            LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_region, source_pos, source_inside, source_detect])
            LSTM_runner.logDataInitialize([source_concentration, water_flow_force, centroids])
            LSTM_runner.learn(num_epochs = 100, bsize = 80, shown = False)
            
            print("loss= {:.2f}, acc= {:.2f}, dis_in= {:.2f}, dis_in_no_detect= {:.2f}, dis_out= {:.2f}, dis_out_no_detect= {:.2f}, ang_in= {:.2f}, ang_in_no_detect= {:.2f}, ang_out= {:.2f}, ang_out_no_detect= {:.2f}".format(\
                LSTM_runner.test_log.getCurrent('conditional_loss'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('error_distance_inside'), \
                LSTM_runner.test_log.getCurrent('error_distance_inside_no_detect'), LSTM_runner.test_log.getCurrent('error_distance_outside'), LSTM_runner.test_log.getCurrent('error_distance_outside_no_detect'),\
                    LSTM_runner.test_log.getCurrent('error_angle_inside'), LSTM_runner.test_log.getCurrent('error_angle_inside_no_detect'), LSTM_runner.test_log.getCurrent('error_angle_outside'),\
                        LSTM_runner.test_log.getCurrent('error_angle_outside_no_detect')))

tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
16
../output/ReportModel/stochastic/reinforce/log/train_log/model_5_False_0.txt 

loss= 109.56, acc= 0.68, dis_in= 11.12, dis_in_no_detect= 5.79, dis_out= 103.22, dis_out_no_detect= 121.60, ang_in= 1.39, ang_in_no_detect= 0.76, ang_out= 0.49, ang_out_no_detect= 0.85


In [9]:
# T_thresholds = [0] #, 1, 1]
# T_ranges = [ 5] #, 20, 30]

new_losses = [False, True]
regularizer = [0, 0.01]
ups = [3, 2, 1]
# T_thresholds = [1]
# T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]

for up in ups:
    getData(variables, map_x * (2**((up-1))))
    for new_loss in new_losses:
        for regularizer_constant in regularizer:
            model_arg['up'] = up
            runner_arg["regularizer_constant"] = regularizer_constant
            runner_arg["new_loss"] = new_loss
            runner_arg["folder_path"] = "../output/ReportModel/stochastic/"
            runner_arg["name"] = "model_{}_{}_{}".format(up*5,new_loss, regularizer_constant)
            LSTM_model = GeneratorPredict(env_arg, model_arg).to(DEVICE)
            LSTM_runner = PytorchRunner(runner_arg=runner_arg)
            LSTM_runner.modelInitialize(LSTM_model)
            LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_region, source_pos, source_inside, source_detect])
            LSTM_runner.logDataInitialize([source_concentration, water_flow_force, centroids])
            LSTM_runner.learn(num_epochs = 100, bsize = 80, shown = False)
            
            print("loss= {:.2f}, acc= {:.2f}, dis_in= {:.2f}, dis_in_no_detect= {:.2f}, dis_out= {:.2f}, dis_out_no_detect= {:.2f}, ang_in= {:.2f}, ang_in_no_detect= {:.2f}, ang_out= {:.2f}, ang_out_no_detect= {:.2f}".format(\
                LSTM_runner.test_log.getCurrent('conditional_loss'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('error_distance_inside'), \
                LSTM_runner.test_log.getCurrent('error_distance_inside_no_detect'), LSTM_runner.test_log.getCurrent('error_distance_outside'), LSTM_runner.test_log.getCurrent('error_distance_outside_no_detect'),\
                    LSTM_runner.test_log.getCurrent('error_angle_inside'), LSTM_runner.test_log.getCurrent('error_angle_inside_no_detect'), LSTM_runner.test_log.getCurrent('error_angle_outside'),\
                        LSTM_runner.test_log.getCurrent('error_angle_outside_no_detect')))

tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
22
../output/ReportModel/stochastic/log/train_log/model_15_False_0.txt 

loss= 43.19, acc= 0.16, dis_in= 22.54, dis_in_no_detect= 70.76, dis_out= 52.07, dis_out_no_detect= 74.61, ang_in= 0.22, ang_in_no_detect= 0.80, ang_out= 0.09, ang_out_no_detect= 0.19
tensor(1.) tensor(1.)
tensor(1.) tensor(1.)
13
../output/ReportModel/stochastic/log/train_log/model_15_False_0.01.txt 

loss= 45.13, acc= 0.14, dis_in= 23.73, dis_in_no_detect= 70.49, dis_out= 52.57, dis_out_no_detect= 78.82, ang_in= 0.23, ang_in_no_detect= 0.80, ang_out= 0.09, ang_out_no_detect= 0.19
tensor(0.6353) tensor(1.)
tensor(0.6361) tensor(1.)
19
../output/ReportModel/stochastic/log/train_log/model_15_True_0.txt 

loss= 39.15, acc= 0.17, dis_in= 24.41, dis_in_no_detect= 86.09, dis_out= 57.83, dis_out_no_detect= 77.28, ang_in= 0.23, ang_in_no_detect= 0.86, ang_out= 0.09, ang_out_no_detect= 0.21
tensor(0.6353) tensor(1.)
tensor(0.6361) tensor(1.)
27
../output/ReportModel/stochastic/lo

In [8]:
#old loss + regularizer: train: c_loss = 0.670, c_acc = 0.508, acc = 0.508, hl_acc = 0.580| test: c_loss = 0.576, c_acc = 0.565, acc = 0.565, hl_acc = 0.638
#new loss + regularizer: train: c_loss = 0.669, c_acc = 0.525, acc = 0.429, hl_acc = 0.495| test: c_loss = 0.610, c_acc = 0.522, acc = 0.460, hl_acc = 0.514

#old_loss:
#new_loss:

T_thresholds = [0] #, 1, 1]
T_ranges = [ 5] #, 20, 30]

# T_thresholds = [1]
# T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]
for i in range(len(T_thresholds)):
    model_arg["T_range"] = T_ranges[i]
    model_arg["T_threshold"] = T_thresholds[i]
    runner_arg["folder_path"] = "../output/ReportModel/stochastic/{}outOf{}/".format(T_thresholds[i], T_ranges[i])
    LSTM_model = GeneratorPredict(env_arg, model_arg).to(DEVICE)
    LSTM_runner = PytorchRunner(runner_arg=runner_arg)
    LSTM_runner.modelInitialize(LSTM_model)
    LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_map_position, source_map_position_index])
    LSTM_runner.logDataInitialize([source_concentration, water_flow_force])
    LSTM_runner.learn(num_epochs = 100, bsize = 100, shown = True)
    print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))
    

tensor(1.) tensor(1.)
tensor(1.) tensor(1.)


train: c_loss = 1.013, c_acc = 0.304, acc = 0.304, hl_acc = 0.295| test: c_loss = 2.343, c_acc = 0.041, acc = 0.041, hl_acc = 0.041: 100%|██████████| 144/144 [00:44<00:00,  3.21it/s]
train: c_loss = 0.811, c_acc = 0.390, acc = 0.390, hl_acc = 0.435| test: c_loss = 0.901, c_acc = 0.332, acc = 0.332, hl_acc = 0.336: 100%|██████████| 144/144 [00:45<00:00,  3.18it/s]
train: c_loss = 0.763, c_acc = 0.424, acc = 0.424, hl_acc = 0.475| test: c_loss = 0.729, c_acc = 0.436, acc = 0.436, hl_acc = 0.475: 100%|██████████| 144/144 [00:44<00:00,  3.24it/s]
train: c_loss = 0.769, c_acc = 0.474, acc = 0.474, hl_acc = 0.533| test: c_loss = 0.682, c_acc = 0.477, acc = 0.477, hl_acc = 0.531: 100%|██████████| 144/144 [00:44<00:00,  3.21it/s]
train: c_loss = 0.749, c_acc = 0.458, acc = 0.458, hl_acc = 0.544| test: c_loss = 0.661, c_acc = 0.497, acc = 0.497, hl_acc = 0.551: 100%|██████████| 144/144 [00:44<00:00,  3.22it/s]
train: c_loss = 0.733, c_acc = 0.499, acc = 0.499, hl_acc = 0.552| test: c_loss = 0.6

19
../output/ReportModel/stochastic/0outOf5/log/train_log/LSTM_0.562920.txt 

0.527041, 0.565292, 0.565292, 0.637636


In [ ]:
#c_loss = 0.755, c_acc = 0.506, acc = 0.425, hl_acc = 0.468| test: c_loss = 0.727, c_acc = 0.439, acc = 0.387, hl_acc = 0.411
T_thresholds = [0] #, 1, 1]
T_ranges = [ 5] #, 20, 30]

# T_thresholds = [1]
# T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]
for i in range(len(T_thresholds)):
    model_arg["T_range"] = T_ranges[i]
    model_arg["T_threshold"] = T_thresholds[i]
    runner_arg["folder_path"] = "../output/ReportModel/stochastic/{}outOf{}/".format(T_thresholds[i], T_ranges[i])
    LSTM_model = GeneratorPredict(env_arg, model_arg).to(DEVICE)
    LSTM_runner = PytorchRunner(runner_arg=runner_arg)
    LSTM_runner.modelInitialize(LSTM_model)
    LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_map_position, source_map_position_index])
    LSTM_runner.logDataInitialize([source_concentration, water_flow_force])
    LSTM_runner.learn(num_epochs = 100, bsize = 100, shown = True)
    print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))
    

tensor(0.6689) tensor(1.)
tensor(0.8347) tensor(1.)


train: c_loss = 1.221, c_acc = 0.278, acc = 0.230, hl_acc = 0.215| test: c_loss = 2.420, c_acc = 0.039, acc = 0.039, hl_acc = 0.039: 100%|██████████| 144/144 [00:42<00:00,  3.35it/s]
train: c_loss = 0.990, c_acc = 0.389, acc = 0.317, hl_acc = 0.338| test: c_loss = 0.963, c_acc = 0.300, acc = 0.274, hl_acc = 0.272: 100%|██████████| 144/144 [00:43<00:00,  3.34it/s]
train: c_loss = 0.862, c_acc = 0.427, acc = 0.360, hl_acc = 0.397| test: c_loss = 0.777, c_acc = 0.410, acc = 0.367, hl_acc = 0.398: 100%|██████████| 144/144 [00:43<00:00,  3.35it/s]
train: c_loss = 0.773, c_acc = 0.456, acc = 0.374, hl_acc = 0.429| test: c_loss = 0.712, c_acc = 0.459, acc = 0.409, hl_acc = 0.448: 100%|██████████| 144/144 [00:44<00:00,  3.26it/s]
train: c_loss = 0.932, c_acc = 0.402, acc = 0.352, hl_acc = 0.365| test: c_loss = 0.693, c_acc = 0.468, acc = 0.416, hl_acc = 0.461: 100%|██████████| 144/144 [00:44<00:00,  3.22it/s]
train: c_loss = 0.783, c_acc = 0.476, acc = 0.369, hl_acc = 0.388| test: c_loss = 0.7

In [28]:
T_thresholds = [0, 1, 1] #, 1, 1]
T_ranges = [ 5, 5, 10] #, 20, 30]

# T_thresholds = [1]
# T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]
for i in range(len(T_thresholds)):
    model_arg["T_range"] = T_ranges[i]
    model_arg["T_threshold"] = T_thresholds[i]
    runner_arg["folder_path"] = "../output/ReportModel/stochastic/{}outOf{}/".format(T_thresholds[i], T_ranges[i])
    LSTM_model = GeneratorPredict(env_arg, model_arg).to(DEVICE)
    LSTM_runner = PytorchRunner(runner_arg=runner_arg)
    LSTM_runner.modelInitialize(LSTM_model)
    LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_map_position, source_map_position_index])
    LSTM_runner.logDataInitialize([source_concentration, water_flow_force])
    LSTM_runner.learn(num_epochs = 100, bsize = 100, shown = True)
    print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))
    

tensor(1.) tensor(1.)
tensor(1.) tensor(1.)


train: c_loss = 19.205, c_acc = 0.631, acc = 0.631, hl_acc = 0.722| test: c_loss = 19.959, c_acc = 0.039, acc = 0.039, hl_acc = 0.039: 100%|██████████| 121/121 [00:36<00:00,  3.31it/s]
train: c_loss = 18.499, c_acc = 0.677, acc = 0.677, hl_acc = 0.753| test: c_loss = 18.050, c_acc = 0.615, acc = 0.615, hl_acc = 0.685: 100%|██████████| 121/121 [00:39<00:00,  3.06it/s]
train: c_loss = 19.112, c_acc = 0.705, acc = 0.705, hl_acc = 0.808| test: c_loss = 17.951, c_acc = 0.662, acc = 0.662, hl_acc = 0.736: 100%|██████████| 121/121 [00:38<00:00,  3.14it/s]
train: c_loss = 18.001, c_acc = 0.695, acc = 0.695, hl_acc = 0.773| test: c_loss = 17.922, c_acc = 0.677, acc = 0.677, hl_acc = 0.754: 100%|██████████| 121/121 [00:37<00:00,  3.23it/s]
train: c_loss = 18.747, c_acc = 0.708, acc = 0.708, hl_acc = 0.765| test: c_loss = 17.867, c_acc = 0.684, acc = 0.684, hl_acc = 0.765: 100%|██████████| 121/121 [00:37<00:00,  3.20it/s]
train: c_loss = 18.414, c_acc = 0.723, acc = 0.723, hl_acc = 0.801| test: c

10
../output/ReportModel/stochastic/0outOf5/log/train_log/LSTM_17.822763.txt 

0.706673, 0.692641, 0.692641, 0.772281
tensor(0.2039) tensor(1.)
tensor(0.2124) tensor(1.)


train: c_loss = 9.569, c_acc = 0.521, acc = 0.385, hl_acc = 0.372| test: c_loss = 11.995, c_acc = 0.039, acc = 0.040, hl_acc = 0.040: 100%|██████████| 121/121 [00:35<00:00,  3.38it/s] 
train: c_loss = 11.442, c_acc = 0.578, acc = 0.428, hl_acc = 0.403| test: c_loss = 10.015, c_acc = 0.563, acc = 0.419, hl_acc = 0.399: 100%|██████████| 121/121 [00:36<00:00,  3.34it/s]
train: c_loss = 10.973, c_acc = 0.639, acc = 0.442, hl_acc = 0.395| test: c_loss = 9.946, c_acc = 0.568, acc = 0.412, hl_acc = 0.387:  58%|█████▊    | 70/121 [00:21<00:15,  3.28it/s]

In [9]:
T_thresholds = [1, 0, 1, 1, 1]
T_ranges = [10, 5, 5, 20, 30]

T_thresholds = [1]
T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]
for i in range(len(T_thresholds)):
    model_arg["T_range"] = T_ranges[i]
    model_arg["T_threshold"] = T_thresholds[i]
    runner_arg["folder_path"] = "../output/ReportModel/stochastic/{}outOf{}/".format(T_thresholds[i], T_ranges[i])
    LSTM_model = LSTMOnGamma(env_arg, model_arg).to(DEVICE)
    LSTM_runner = PytorchRunner(runner_arg=runner_arg)
    LSTM_runner.modelInitialize(LSTM_model)
    LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_map_position, source_map_position_index])
    LSTM_runner.logDataInitialize([source_concentration, water_flow_force])
    LSTM_runner.learn(num_epochs = 100, bsize = 100, shown = True)
    print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))
    

tensor(0.2921) tensor(1.)
tensor(0.2902) tensor(1.)


train: c_loss = 1.357, c_acc = 0.210, acc = 0.083, hl_acc = 0.048| test: c_loss = 2.631, c_acc = 0.041, acc = 0.040, hl_acc = 0.040: 100%|██████████| 144/144 [00:38<00:00,  3.74it/s]
train: c_loss = 1.161, c_acc = 0.259, acc = 0.133, hl_acc = 0.097| test: c_loss = 1.285, c_acc = 0.194, acc = 0.076, hl_acc = 0.050: 100%|██████████| 144/144 [00:38<00:00,  3.75it/s]
train: c_loss = 0.991, c_acc = 0.316, acc = 0.140, hl_acc = 0.111| test: c_loss = 1.145, c_acc = 0.264, acc = 0.124, hl_acc = 0.095: 100%|██████████| 144/144 [00:38<00:00,  3.74it/s]
train: c_loss = 1.074, c_acc = 0.330, acc = 0.154, hl_acc = 0.117| test: c_loss = 1.083, c_acc = 0.295, acc = 0.147, hl_acc = 0.119: 100%|██████████| 144/144 [00:38<00:00,  3.69it/s]
train: c_loss = 0.917, c_acc = 0.345, acc = 0.171, hl_acc = 0.134| test: c_loss = 1.062, c_acc = 0.302, acc = 0.148, hl_acc = 0.120: 100%|██████████| 144/144 [00:39<00:00,  3.65it/s]
train: c_loss = 0.904, c_acc = 0.333, acc = 0.133, hl_acc = 0.104| test: c_loss = 1.0

25
../output/ReportModel/stochastic/1outOf10/log/train_log/LSTM_0.931325.txt 

0.384118, 0.372247, 0.199539, 0.178736


In [ ]:
T_thresholds = [1, 0, 1, 1, 1]
T_ranges = [10, 5, 5, 20, 30]

T_thresholds = [1]
T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]
for i in range(len(T_thresholds)):
    model_arg["T_range"] = T_ranges[i]
    model_arg["T_threshold"] = T_thresholds[i]
    runner_arg["folder_path"] = "../output/ReportModel/stochastic/{}outOf{}/".format(T_thresholds[i], T_ranges[i])
    LSTM_model = LSTMPredict(env_arg, model_arg).to(DEVICE)
    LSTM_runner = PytorchRunner(runner_arg=runner_arg)
    LSTM_runner.modelInitialize(LSTM_model)
    LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_map_position, source_map_position_index])
    LSTM_runner.logDataInitialize([source_concentration, water_flow_force])
    LSTM_runner.learn(num_epochs = 100, bsize = 100, shown = True)
    print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))
    

In [7]:
T_thresholds = [1, 0, 1, 1, 1]
T_ranges = [10, 5, 5, 20, 30]

T_thresholds = [1]
T_ranges = [10]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]
for i in range(len(T_thresholds)):
    model_arg["T_range"] = T_ranges[i]
    model_arg["T_threshold"] = T_thresholds[i]
    runner_arg["folder_path"] = "../output/ReportModel/stochastic/{}outOf{}/".format(T_thresholds[i], T_ranges[i])
    LSTM_model = LSTMPredict(env_arg, model_arg).to(DEVICE)
    LSTM_runner = PytorchRunner(runner_arg=runner_arg)
    LSTM_runner.modelInitialize(LSTM_model)
    LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_map_position, source_map_position_index])
    LSTM_runner.logDataInitialize([source_concentration, water_flow_force])
    LSTM_runner.learn(num_epochs = 100, bsize = 100, shown = True)
    print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))
    

tensor(0.2921) tensor(0.8345)
tensor(0.2902) tensor(0.8347)


train: c_loss = 1.176, c_acc = 0.237, acc = 0.115, hl_acc = 0.075| test: c_loss = 2.657, c_acc = 0.040, acc = 0.040, hl_acc = 0.041: 100%|██████████| 144/144 [00:53<00:00,  2.68it/s]
train: c_loss = 1.081, c_acc = 0.290, acc = 0.172, hl_acc = 0.125| test: c_loss = 1.214, c_acc = 0.239, acc = 0.118, hl_acc = 0.081: 100%|██████████| 144/144 [00:55<00:00,  2.61it/s]
train: c_loss = 1.213, c_acc = 0.281, acc = 0.166, hl_acc = 0.117| test: c_loss = 1.087, c_acc = 0.299, acc = 0.173, hl_acc = 0.131: 100%|██████████| 144/144 [00:53<00:00,  2.71it/s]
train: c_loss = 1.013, c_acc = 0.322, acc = 0.168, hl_acc = 0.108| test: c_loss = 1.034, c_acc = 0.324, acc = 0.193, hl_acc = 0.151: 100%|██████████| 144/144 [00:54<00:00,  2.65it/s]
train: c_loss = 0.968, c_acc = 0.350, acc = 0.203, hl_acc = 0.154| test: c_loss = 1.012, c_acc = 0.334, acc = 0.193, hl_acc = 0.149: 100%|██████████| 144/144 [00:54<00:00,  2.66it/s]
train: c_loss = 0.972, c_acc = 0.353, acc = 0.201, hl_acc = 0.162| test: c_loss = 0.9

KeyboardInterrupt: 

In [8]:
LSTM_runner.saveModel()

../output/ReportModel/stochastic/1outOf10/log/train_log/LSTM_0.246335.txt 



In [9]:
print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))

0.307540, 0.300878, 0.193734, 0.194857


In [9]:
print("{:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log_nolimit.getCurrent('accuracy')))
print("{:3f}".format(LSTM_runner.train_log.getCurrent('accuracy')))
print("{:3f}".format(LSTM_runner.test_log.getCurrent('accuracy')))
print("{:3f}".format(LSTM_runner.test_log_nolimit.getCurrent('accuracy')))


0.168703
0.168031
0.168031


In [10]:
print("{:3f}".format(LSTM_runner.train_log.getCurrent('loss')))
print("{:3f}".format(LSTM_runner.test_log.getCurrent('loss')))
print("{:3f}".format(LSTM_runner.test_log_nolimit.getCurrent('loss')))


1.291019
1.293081
1.293081


In [4]:
import torch
a = [ torch.ones(3, 5), torch.ones(3, 5), torch.ones(4, 5)]
torch.cat(a, dim = 0).shape

torch.Size([10, 5])

In [ ]:
T_thresholds = [1, 0, 1] #, 1, 1]
T_ranges = [10, 5, 5] #, 20, 30]

# T_thresholds = [1]
# T_ranges = [10]
k_taps = [5, 10] #, 15, 21]
# T_thresholds = [1, 1]
# T_ranges = [30, 20]

model_arg["T_range"] = 10
model_arg["T_threshold"] = 1
# for i in range(len(T_thresholds)):
for k_tap in k_taps:
    # model_arg["T_range"] = T_ranges[i]
    # model_arg["T_threshold"] = T_thresholds[i]
    model_arg["k_tap"] = k_tap
    runner_arg["folder_path"] = "../output/ReportModel/stochastic/{}outOf{}/".format(1, 10)
    LSTM_model = LSTMOnGamma(env_arg, model_arg).to(DEVICE)
    LSTM_runner = PytorchRunner(runner_arg=runner_arg)
    LSTM_runner.modelInitialize(LSTM_model)
    LSTM_runner.dataInitialize([X_input, agent_action, distance_map, source_map_position, source_map_position_index])
    LSTM_runner.logDataInitialize([source_concentration, water_flow_force])
    LSTM_runner.learn(num_epochs = 100, bsize = 100, shown = False)
    print("{:3f}, {:3f}, {:3f}, {:3f}".format(LSTM_runner.train_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('conditional_accuracy'), LSTM_runner.test_log.getCurrent('accuracy'), LSTM_runner.test_log.getCurrent('half_last_accuracy')))
    